# Univariate Models

In [ ]:
# TODO: look at residuals
# Line plot of residuals
# residuals_ets = y_true_ets - y_pred_ets
# plt.figure(figsize=(10, 4))
# plt.plot(y_true_ets.index, residuals_ets, label="Residuals")
# plt.axhline(
#     0, color="red", linestyle="--", linewidth=2, label="Zero Residuals"
# )
# plt.xlabel("Time")
# plt.ylabel("Residuals")
# plt.legend()
# plt.title(f"{col_ets} - Residuals (ETS Model)")
# plt.show()

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

from src.data_preprocessing.data_loader import load_data, time_split
from src.modeling.evaluation import smape, mae
from src.modeling.univariate_modeling import get_best_cv_model

In [ ]:
DATA_DIR = (
    Path("..")
    / ".."
    / ".."
    / "hfactory_magic_folders"
    / "plastic_cost_prediction"
    / "data"
)
MAIN_FILE = "PA6_cleaned_dataset.csv"

In [ ]:
df = load_data(DATA_DIR / MAIN_FILE)

In [ ]:
models = get_best_cv_model(df)

In [ ]:
color_dict = {
    "baseline": "grey",
    "ARIMA": "orange",
    "ETS": "blue",
    "XGB": "green",
}

for col in models:
    series = df[col].copy().dropna()
    y_train = series.iloc[:-9]
    y_test = series.iloc[-9:]

    plt.figure(figsize=(10, 6))
    plt.plot(y_train.index, y_train, label="Training data", color="black")
    plt.plot(y_test.index, y_test, label="Test data", color="red")

    for model in models[col]:
        if model not in ["selected", "XGB"]:
            plt.plot(
                y_test.index,
                models[col][model]["preds"],
                label=model,
                color=color_dict[model],
            )
        elif model == "XGB":
            plt.plot(
                y_test.index[2::3],
                models[col][model]["preds"],
                ".",
                label=model,
                color=color_dict[model],
            )

    plt.xlabel("Time")
    plt.ylabel("Values")
    plt.legend()
    plt.title(
        f"{col} - Actual vs Predictions \n Final Model: {models[col]['selected']}"
    )
    plt.show()

## Best Univariate Model for the Target

We want to invesigate the average performance across folds for each time horizon (3-6-9 months) of the best model.

In [ ]:
target = "best_price_compound"

In [ ]:
series = df[target].copy().dropna()
target_model = models[target][models[target]["selected"]]
spl = time_split(series)
preds_3, preds_6, preds_9 = [], [], []
target_3, target_6, target_9 = [], [], []

for train_idx, test_idx in spl:
    train = series.iloc[train_idx]
    test = series.iloc[test_idx]
    model = ARIMA(
        train,
        order=target_model["order"],
        seasonal_order=target_model["seasonal_order"],
    )
    model_fit = model.fit()
    preds = model_fit.predict(start=test.index[0], end=test.index[-1])
    preds_3.append(preds[2])
    preds_6.append(preds[5])
    preds_9.append(preds[8])
    target_3.append(test[2])
    target_6.append(test[5])
    target_9.append(test[8])

In [ ]:
smape_3 = smape(np.array(target_3), np.array(preds_3))
smape_6 = smape(np.array(target_6), np.array(preds_6))
smape_9 = smape(np.array(target_9), np.array(preds_9))

mae_3 = mae(np.array(target_3), np.array(preds_3))
mae_6 = mae(np.array(target_6), np.array(preds_6))
mae_9 = mae(np.array(target_9), np.array(preds_9))

In [ ]:
print("3 months:\n", f"- sMAPE: {smape_3:.2f}% \n", f"- MAE: {mae_3:.2f}")
print("6 months:\n", f"- sMAPE: {smape_6:.2f}% \n", f"- MAE: {mae_6:.2f}")
print("9 months:\n", f"- sMAPE: {smape_9:.2f}% \n", f"- MAE: {mae_9:.2f}")